In [1]:
import sys, os
os.environ["CUDA_VISIBLE_DEVICES"]="7"

import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from keras.callbacks import TensorBoard
from keras import backend as K
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()

from cifar10_robust_models import Load_Madry_Model, cifar10_tf_robust_models

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train[:100]
y_train = y_train[:100]
x_test = x_test[:100]
y_test = y_test[:100]
x_train.shape

with tf.Session() as sess:
    keras.backend.set_session(sess)
    model_dir = "CIFAR10_models/Robust_Deep_models/Madry_robust_target_model"
    target_model2 = Load_Madry_Model(tf.Session(), model_dir,bias = 0.5, scale = 255)
    #target_model3 = Load_Madry_Model(tf.Session(), model_dir,bias = 0.5, scale = 255)
    
    input_img = Input(shape=(32, 32, 3))

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)

    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
    
    def custom_loss():

        # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
        def loss(y_true,y_pred):
            target_model2.model.x_input = (y_true+target_model2.bias) * target_model2.scale
            prob_true = tf.nn.softmax(target_model2.model.pre_softmax, axis=1)
            
            target_model2.model.x_input = (y_pred+target_model2.bias) * target_model2.scale
            prob_pred = tf.nn.softmax(target_model2.model.pre_softmax, axis=1)
            
            return tf.keras.backend.binary_crossentropy(prob_true, prob_pred)

        # Return a function
        return loss
    
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss=custom_loss())

    autoencoder.fit(x_train, x_train,
                    epochs=1,
                    batch_size=1,
                    shuffle=True,
                    validation_data=(x_test, x_test),
                    callbacks=[TensorBoard(log_dir='/tmp/autoencoder')],
                    )

    autoencoder.summary()


Using TensorFlow backend.



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from CIFAR10_models/Robust_Deep_models/Madry_robust_target_model/checkpoint-70000


ValueError: An operation has `None` for gradient. Please make sure that all of your ops have a gradient defined (i.e. are differentiable). Common ops without gradient: K.argmax, K.round, K.eval.

In [ ]:
with sess:
    
    
    input_img = Input(shape=(32, 32, 3))

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)

    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
    
    def custom_loss():

        # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
        def loss(y_true,y_pred):
            prob_pred = tf.py_function(target_model2.predict_prob_temp, inp=[y_pred], Tout=tf.float32)
            prob_true = tf.py_function(target_model2.predict_prob_temp, inp=[y_true], Tout=tf.float32)
            tf.print(y_true, output_stream=sys.stdout)
            return K.mean(K.square(y_true - y_pred), axis=-1)

        # Return a function
        return loss
    
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss=custom_loss())

    autoencoder.fit(x_train, x_train,
                    epochs=1,
                    batch_size=1,
                    shuffle=True,
                    validation_data=(x_test, x_test),
                    callbacks=[TensorBoard(log_dir='/tmp/autoencoder')],
                    )

    autoencoder.summary()


In [ ]:
with tf.Session() as sess:
    keras.backend.set_session(sess)
    model_dir = "CIFAR10_models/Robust_Deep_models/Madry_robust_target_model"
    target_model2 = Load_Madry_Model(sess, model_dir,bias = 0.5, scale = 255)
    
    input_img = Input(shape=(32, 32, 3))

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)

    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
    
    def custom_loss():

        # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
        def loss(y_true,y_pred):
            #cross entropy loss (target_model2(y_pred), target_model2(y_true))
            
            
            
#             prob_pred = tf.py_function(target_model2.predict_prob_temp, inp=[y_pred], Tout=tf.float32)
#             prob_true = tf.py_function(target_model2.predict_prob_temp, inp=[y_true], Tout=tf.float32)
            #tf.print(y_true, output_stream=sys.stdout)
            return K.mean(K.square(target_model2(y_pred) - target_model2(y_true)), axis=-1)

        # Return a function
        return loss
    
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss=custom_loss())

    autoencoder.fit(x_train, x_train,
                    epochs=1,
                    batch_size=1,
                    shuffle=True,
                    validation_data=(x_test, x_test),
                    callbacks=[TensorBoard(log_dir='/tmp/autoencoder')],
                    )

    autoencoder.summary()

In [ ]:
#Encoder model
with sess:
    

In [ ]:
#Decoder model
with sess:
    

In [ ]:
# @tf.custom_gradient
# def py_loss_fn(y_true, y_pred):
#     """ This function takes eager tensors as inputs which can be explicitly
#     converted to np.arrays via EagerTensor.numpy() or implicitly converted
#     by applying numpy operations to them.

#     However, once tf operations are no longer used it means that the function has to
#     implement its own gradient function.
#     """
#     def grad(dy):
#         """ Compute gradients for function inputs.
#             Ignore input[0] (y_true) since that is model.targets[0]
#         """
#         g = np.mean(-dy * np.sign(y_true - y_pred), axis=1)[:, np.newaxis]
#         return None, g
    
#     prob_pred = target_model2.predict_prob_temp(y_pred)
#     prob_true = target_model2.predict_prob_temp(y_true)
#     return cross_entropy(prob_pred, prob_true), grad

# def cross_entropy(p, q):
#     return -sum([p[i]*log(q[i]) for i in range(len(p))])

# def custom_loss():

#     # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
#     def new_loss(y_true,y_pred):
#         prob_pred = tf.py_function(target_model2.predict_prob_temp, inp=[y_pred], Tout=tf.float32)
#         prob_true = tf.py_function(target_model2.predict_prob_temp, inp=[y_true], Tout=tf.float32)
#         tf.print(prob_pred)
#         #l = tf.keras.backend.binary_crossentropy(prob_pred, prob_true)
#         return prob_pred

#     # Return a function
#     return new_loss
with sess:
    

In [ ]:
with sess:
    

In [ ]:
autoencoder.predict(test1)

In [ ]:
#saving model to disk

model_json = autoencoder.to_json()

with open('model_json', 'w') as json_file:
    json_file.write(model_json)
autoencoder.save_weights("model.h5")
print("Saved model to disk..")

In [ ]:
from keras.models import model_from_json

json_file = open('model_json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('model.h5')
print("Loaded model from disk")
decoded_imgs = loaded_model.predict(x_test)

In [ ]:
decoded_imgs

In [ ]:
!pip list


In [ ]:
def new_loss(y_true,y_pred):
        return binary_crossentropy(classifer(y_pred),classifer(y_true))- constant * min(np.linalg.norm(y_true,y_pred)**2,d)

In [ ]:
from . import cifar10_robust_models.Load_MadryM

In [ ]:
import sys
print(sys.path)

In [ ]:
x_test.shape

In [ ]:
target_model.shape

In [ ]:
type(x_test)

In [ ]:
test1 = np.ndarray(shape=(1,32,32,3), dtype=float)

In [ ]:
test1[0] = x_test[1]

In [ ]:
test1

In [ ]:
test1.shape

In [ ]:
len(test1)

In [ ]:
len(x_test)

In [ ]:
res = target_model.predict_prob(test1)

In [ ]:
res.shape

In [ ]:
len(y_test) - np.sum(res == y_test)

In [ ]:
y_test.shape

In [ ]:
res.shape

In [ ]:
count = 0
for i in range(10000):
    if y_test[i][0] == res[i]:
        count +=1
print(count*100.00/10000)

In [ ]:
from numpy import load


In [ ]:
data = load('data_numpy_image.npy')

In [ ]:
data

In [ ]:
from PIL import Image


In [ ]:
img = Image.fromarray(data[0]*255, 'RGB')
img.save('my.png')
img.show()

In [ ]:
data[0]*255